In [11]:
from pathlib import Path
import time

import roach

from relbench.data import TaskType
from relbench.datasets import get_dataset_names
from relbench.tasks import get_task_names, get_task

In [4]:
project = "relbench/2024-07-01"
queue = "relbench"

# cache materialized tensorframes

In [13]:
# clear current cache with backup
ts = int(time.time())
home = Path.home()
try:
    Path(f"{home}/.cache/relbench_examples").rename(f"{home}/.cache/relbench_examples.{ts}")
except FileNotFoundError:
    pass

In [15]:
for dataset in get_dataset_names():
    task = get_task_names(dataset)[0]
    task_obj = get_task(dataset, task)
    if task_obj.task_type == TaskType.LINK_PREDICTION:
        script = "gnn_link"
    else:
        script = "gnn_node"
    # epochs=0 throws exception
    cmd = f"python {script}.py --dataset {dataset} --task {task} --epochs 1"
    roach.submit(queue, cmd)

In [18]:
# download all tasks beforehand
for dataset in get_dataset_names():
    for task in get_task_names(dataset):
        py_cmd = (
            f"from relbench.tasks import download_task; "
            f"download_task(\"{dataset}\", \"{task}\")"
        )
        assert "'" not in py_cmd
        cmd = f"python -c '{py_cmd}'"
        roach.submit(queue, cmd)

In [16]:
for seed in range(1):
    for dataset in get_dataset_names():
        if dataset not in ["rel-f1", "rel-trial"]:
            continue
        for task in get_task_names(dataset):
            task_obj = get_task(dataset, task)
            if task_obj.task_type == TaskType.LINK_PREDICTION:
                scripts = [
                    "gnn_link",
                    "idgnn_link",
                    "lightgbm_link_baseline",
                    "link_baseline",
                ]
            else:
                scripts = [
                    "gnn_node",
                    "lightgbm_baseline",
                    "lightgbm_gnn_features_node",
                    "node_baseline",
                ]
            for script in scripts:
                cmd = (
                    f"OMP_NUM_THREADS=8"
                    f" python {script}.py"
                    f" --dataset {dataset}"
                    f" --task {task}"
                    f" --seed {seed}"
                    f" --roach_project {project}"
                )
                requires = (
                    f"test -d ~/.cache/relbench_examples/{dataset} && "
                    f"test -d ~/.cache/relbench/{dataset}/tasks/{task}.zip"
                )
                roach.submit(queue, cmd, requires)